In [1]:
from transformers import Trainer, TrainingArguments
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
from tqdm import tqdm
from datasets import load_dataset, load_metric

In [2]:
model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model.to(device)

def classify_text(text, possible_outputs):
    framed_texts = [f"{text} This text is {output}." for output in possible_outputs]
    encoded_inputs = [tokenizer.encode(t, return_tensors="pt").to(device) for t in framed_texts]
    
    logits_for_outputs = []

    for encoded_input in encoded_inputs:
        with torch.no_grad():
            outputs = model(encoded_input)
            logits = outputs.logits

        logits_for_outputs.append(logits[0, -1, :].squeeze().cpu().numpy())
    
    token_ids = [tokenizer.encode(output)[0] for output in possible_outputs]
    class_logits = [logits[token_id] for logits, token_id in zip(logits_for_outputs, token_ids)]
    return possible_outputs[class_logits.index(max(class_logits))]

possible_classes = ["positive", "negative"]


In [3]:
import torch
from transformers import GPT2Tokenizer, GPT2LMHeadModel

model_name = 'gpt2-medium'
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

model.eval()
model.to('cuda:1' if torch.cuda.is_available() else 'cpu')

def summarize(text, max_length=150):
    prompt = f"Summarize the following text: \"{text}\" Summary: "

    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    input_ids = input_ids.to('cuda:1' if torch.cuda.is_available() else 'cpu')

    with torch.no_grad():
        output = model.generate(input_ids, max_length=max_length, num_return_sequences=1, temperature=0.7, pad_token_id=tokenizer.eos_token_id)

    generated_text = tokenizer.decode(output[0], skip_special_tokens=True)
    summary = generated_text.split("Summary: ")[-1]

    return summary

article = "Artificial intelligence (AI) is intelligence demonstrated by machines, in contrast to the natural intelligence displayed by humans and animals. Leading AI textbooks define the field as the study of \"intelligent agents\": any device that perceives its environment and takes actions that maximize its chance of successfully achieving its goals."
print(summarize(article))

 The term "AI" is used to describe a type of artificial intelligence that is capable of performing tasks that humans cannot perform.


In [59]:
sst2 = load_dataset("glue", "sst2")
train_data = sst2['train']

ground_truth = []
pred_labels = []

for data in tqdm(train_data):
    logits = data['label']
    prediction = classify_text(data['sentence'], possible_classes)
    if prediction == "positive":
        pred_labels.append(1)
    else:
        pred_labels.append(0)
    ground_truth.append(logits)
    

Found cached dataset glue (/home/minhaoj2/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)


  0%|          | 0/3 [00:00<?, ?it/s]

100%|██████████| 67349/67349 [19:40<00:00, 57.07it/s]


In [60]:
from sklearn.metrics import accuracy_score
print(accuracy_score(ground_truth, pred_labels))

0.450608026845239
